<a href="https://colab.research.google.com/github/yilmazabdullah/2d-captcha/blob/main/ESRGAN_4_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ESRGAN for increasing resolution ON VIDEOS!!!**
by HostsServer aka [MSFTserver](https://github.com/MSFTserver)
####**-SEEMS TO ONLY WORK CONSISTENTLY ON CHROME!!**
#### **-DOUBLE CHECK YOUR RUNTIME TYPE IS SET TO GPU**
example: an image with height 500px by width 500px scaled to an image with height 1762px by width 1762px

**You might need to go to Runtime>Restart Runtime**

If you open the folder icon (in the left menu) you will see all your files, the good thing about doing **Runtime>Restart Runtime** is that it keeps all the files you downloaded but Restarts the Memory on the Cloud Computer so you can put the ESRGAN model in memory, compared to **Runtime>Factory Reset Runtime** which Restarts the whole Cloud Computer to a default clear state.


In [ ]:
#@markdown # **1) Download required Libraries**
# # Clone Real-ESRGAN and enter the Real-ESRGAN
from google.colab import drive, files
import os, shutil, subprocess
drive_mounted = False
temp_folder = 'tmp_frames'
result_folder = 'results'
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
%cd ..

In [ ]:
#@markdown # **2.A) Optional:** Mount Google Drive
#@markdown uploading videos is really slow it will be faster to connect your drive and spcify the filename or path the video you want to use here

#@markdown Step 2.B can be Skipped if using google drive!
if os.path.exists('/content/drive') is not True:
  drive.mount('/content/drive')

drive_mounted = True
drive_path = '/video/test.mp4'#@param{type: "string"}
file_name = drive_path.split('/')[-1]
input_path = f'/content/drive/MyDrive/{drive_path}'
print("video path set:\n",input_path)

In [ ]:
#@markdown # **2.B) Run this block and Upload Video using the DIALOG box!**
#@markdown Wait till it loads the video and once it's done run the next block

#@markdown You can drag and drop into choose files button
#@markdown ## SKIP this if you mounted your Google Drive!
if drive_mounted is not True:

  upload_folder = 'upload'

  if os.path.isdir(upload_folder):
      shutil.rmtree(upload_folder)

  os.mkdir(upload_folder)

  # upload images
  uploaded = files.upload()
  file_name = next(iter(uploaded))
  input_path = f'/content/{upload_folder}/{file_name}'
  for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)
else:
  print("drive mounted nothing to do here move along")

In [ ]:
#@markdown # **2.B) Run this block and Upload Video using the DIALOG box!**
#@markdown Wait till it loads the video and once it's done run the next block

#@markdown You can drag and drop into choose files button
#@markdown ## SKIP this if you mounted your Google Drive!
if drive_mounted is not True:

  upload_folder = 'upload'

  if os.path.isdir(upload_folder):
      shutil.rmtree(upload_folder)

  os.mkdir(upload_folder)

  # upload images
  uploaded = files.upload()
  file_name = next(iter(uploaded))
  input_path = f'/content/{upload_folder}/{file_name}'
  for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)
else:
  print("drive mounted nothing to do here move along")

In [ ]:
#@markdown # **3) Extract Frames from Video**

if os.path.isdir(temp_folder):
  shutil.rmtree(temp_folder)

os.mkdir(temp_folder)
print(f'Extracting Frames to: {temp_folder}')
cmd = [
       'ffmpeg',
       '-i',
       input_path,
       '-qscale:v',
       '1',
       '-qmin',
       '1',
       '-qmax',
       '1',
       '-vsync',
       '0',
       f'{temp_folder}/frame_%08d.png'
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    frame_count = len(os.listdir(f'/content/{temp_folder}'))
    print(f"Done, Extracted {frame_count} Frames")


In [ ]:
#@markdown # **4) Run ERSGAN on Extracted Frames!**
%cd /content/Real-ESRGAN
frame_count = len(os.listdir(f'/content/{temp_folder}'))
print(f"Enhancing {frame_count} Frames with ESRGAN...")
cmd = [
    'python',
    'inference_realesrgan.py',
    '-n',
    'RealESRGAN_x4plus',
    '-i',
    f'../{temp_folder}',
    '--outscale',
    '4',
    '--face_enhance'
]
#process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
# stdout, stderr = process.communicate()
# if process.returncode != 0:
#     print(stderr)
#     raise RuntimeError(stderr)
# else:
#     print("Done Extracting Frames")
!{' '.join(cmd)}
print("Done Enhancing Frames")
%cd ..

In [ ]:
#@markdown # **5) Recreate Video From Enhanced Frames!**
import subprocess
import os
import shutil

# Önce dosya yapısını kontrol edelim
print("📁 Mevcut dosya yapısını kontrol ediyorum...")

# Frame'lerin olduğu klasörü bul
possible_folders = ['output_frames', 'results', 'Real-ESRGAN/results', 'enhanced_frames']
frame_folder = None

for folder in possible_folders:
    if os.path.exists(f'/content/{folder}') and any(f.endswith('.png') for f in os.listdir(f'/content/{folder}')):
        frame_folder = f'/content/{folder}'
        print(f"✅ Frame'ler bulundu: {frame_folder}")
        break

if frame_folder is None:
    print("❌ Frame klasörü bulunamadı! Mevcut klasörler:")
    !find /content -name "*.png" -type f | head -10
    raise RuntimeError("Frame klasörü bulunamadı")

# Frame sayısını ve dosya isimlerini kontrol et
frame_files = sorted([f for f in os.listdir(frame_folder) if f.endswith('.png')])
frame_count = len(frame_files)
print(f"📊 Bulunan frame sayısı: {frame_count}")
print(f"📝 İlk 5 frame: {frame_files[:5]}")

# Frame dosya isimlendirmesini analiz et
first_frame = frame_files[0]
if 'frame_' in first_frame and '_out' in first_frame:
    frame_pattern = "frame_%08d_out.png"
elif 'frame_' in first_frame:
    frame_pattern = "frame_%08d.png"
else:
    frame_pattern = "%08d.png"

print(f"🔍 Frame pattern: {frame_pattern}")

# Çıktı klasörünü hazırla
result_folder = "final_videos"
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder, exist_ok=True)

# Video parametreleri
fps = 15  #@param {type: "number"}
file_name = "enhanced_video.mp4"

print(f"🎥 {frame_count} Frame'i videoya dönüştürüyorum...")

# DÜZELTİLMİŞ FFmpeg komutu
cmd = [
    'ffmpeg',
    '-y',  # Overwrite without asking
    '-framerate', str(fps),
    '-i', f'{frame_folder}/{frame_pattern}',
    '-c:v', 'libx264',
    '-pix_fmt', 'yuv420p',
    '-crf', '18',
    '-preset', 'medium',
    '-r', str(fps),
    f'{result_folder}/{file_name}'
]

print("🔧 Çalıştırılan komut:", " ".join(cmd))

# FFmpeg'i çalıştır
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

if process.returncode != 0:
    print("❌ Hata oluştu:")
    print(stderr.decode())

    # Alternatif deneyelim
    print("🔄 Alternatif yöntem deniyorum...")

    # Frame'leri tek tek listeleyerek
    with open('frame_list.txt', 'w') as f:
        for frame_file in sorted(os.listdir(frame_folder)):
            if frame_file.endswith('.png'):
                f.write(f"file '{os.path.join(frame_folder, frame_file)}'\n")
                f.write(f"duration {1/fps}\n")

    alt_cmd = [
        'ffmpeg', '-y',
        '-f', 'concat',
        '-safe', '0',
        '-i', 'frame_list.txt',
        '-c:v', 'libx264',
        '-pix_fmt', 'yuv420p',
        '-crf', '18',
        '-preset', 'medium',
        f'{result_folder}/alt_{file_name}'
    ]

    alt_process = subprocess.Popen(alt_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    alt_stdout, alt_stderr = alt_process.communicate()

    if alt_process.returncode == 0:
        print("✅ Alternatif yöntemle video oluşturuldu!")
    else:
        print("❌ Alternatif yöntem de başarısız:")
        print(alt_stderr.decode())
        raise RuntimeError("Video oluşturulamadı")

else:
    print("✅ Video başarıyla oluşturuldu!")

# Sonucu kontrol et
if os.path.exists(f'{result_folder}/{file_name}'):
    video_size = os.path.getsize(f'{result_folder}/{file_name}') / (1024 * 1024)
    print(f"📊 Video boyutu: {video_size:.1f} MB")

    # Video bilgilerini göster
    info_cmd = [
        'ffprobe', '-v', 'error',
        '-select_streams', 'v:0',
        '-show_entries', 'stream=width,height,r_frame_rate,duration,bit_rate',
        '-of', 'csv=p=0',
        f'{result_folder}/{file_name}'
    ]

    info_result = subprocess.run(info_cmd, capture_output=True, text=True)
    if info_result.returncode == 0:
        info = info_result.stdout.strip().split(',')
        print(f"📐 Çözünürlük: {info[0]}x{info[1]}")
        print(f"🎞️  FPS: {info[2]}")
        print(f"⏱️  Süre: {float(info[3]):.1f}s")
        print(f"📶 Bitrate: {int(info[4])/1000:.0f} kbps")

print("🎉 Video oluşturma işlemi tamamlandı!")

In [ ]:
#@markdown # **Optional**: Unmount Google Drive
#@markdown might be useful if you decide you want to upload on here instead
drive_mounted = False
drive.flush_and_unmount()